# Results Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

from datetime import datetime
import time

import json
import jinja2

from PIL import Image

from common import Printable, projdir
from event import Event
from constants import *

## Process Years

Process all available years

In [2]:
events = {}
existingNames = {}

def processEvents():
    '''Process all events from 1998 onwards'''

    eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '[1-2][0-9][0-9][0-9]')))
    currYear = os.path.basename(eventPaths[-1])[:4]
    for eventPath in eventPaths:
        eventYear = os.path.basename(eventPath)[:4]

        if eventYear == currYear and appConfig['Latest']['Refresh'] or \
                eventYear != currYear and appConfig['History']['Refresh']:
            if eventYear == currYear:
                verbosity = appConfig['Latest']['Verbosity']
            else:
                verbosity = appConfig['History']['Verbosity']

            event = Event(eventPath, appConfig, existingNames=existingNames, verbosity=verbosity)
            event.processEvent()

            events[event.year] = event


def getBanner():
    '''Get banner details'''
    
    bannerFn = appConfig['Event']['Banner']
    banner = {}

    banner['src'] = os.path.join('..', 'img', bannerFn)
    banner['alt'] = appConfig['Event']['Name']

    with Image.open(os.path.join(projdir, 'docs', 'img', bannerFn)) as bannerImage:
        banner['width'], banner['height'] = bannerImage.size

    return banner


def getImageDetails(config, imageId):
    '''Get image details'''
    
    imageDetails = {}

    if 'Images' in config:
        images = config['Images']
        
        if imageId in images:
            image = images[imageId]
            imageFn = image['File']

            imageDetails['src'] = os.path.join('..', 'img', imageFn)
            imageDetails['alt'] = image['Name']
            try:
                imageDetails['href'] = image['Link']
            except:
                pass

            with Image.open(os.path.join(projdir, 'docs', 'img', imageFn)) as bannerImage:
                imageDetails['width'], imageDetails['height'] = bannerImage.size

    return imageDetails


def createIndex():
    '''Create index for available years'''

    # Try legacy banner configuration first, if absent then use the latest image configuration
    try:
        banner = getBanner()
        logo = None
        sponsors = None
    except:
        banner = getImageDetails(appConfig, 'Banner')
        logo = getImageDetails(appConfig, 'Logo')
        sponsors = getImageDetails(appConfig, 'Sponsors')
    
    eventName = appConfig['Event']['Name']
    pageTitle = eventName
    pageDescription = eventName

    subDirs = []
    for eventId in reversed(events):
        event = events[eventId]

        if len(event.entrants) > 0 or event.comment:
            eventLogo = getImageDetails(event.eventConfig, 'Logo')
            href = os.path.join(str(eventId), 'index.html')
            eventTitle = eventId

            fastest, speed = event.getFastest()
            
            subDir = {'id': eventId, 'logo': eventLogo, 'href': href, 'label': eventTitle, 'fastest': fastest, 'speed': speed}
            subDirs.append(subDir)

    htmlDir = os.path.join(projdir, 'docs', EVENTS_DIR)
    htmlFile = os.path.join(htmlDir, 'index.html')

    templatePath = os.path.join(projdir, 'python', 'templates')
    templateLoader = jinja2.FileSystemLoader(searchpath=templatePath)
    templateEnv = jinja2.Environment(loader=templateLoader, autoescape=True, trim_blocks=True, lstrip_blocks=True)

    template = templateEnv.get_template("index.html")
    html = template.render(
                    eventName=eventName,
                    banner=banner,
                    logo=logo,
                    sponsors=sponsors,
                    className=None,
                    pageTitle=pageTitle,
                    pageComment=None,
                    pageDescription=pageDescription,
                    periodDate=None,
                    filesTitle=None,
                    subDirTitle='Events',
                    cssPath=os.path.join('..', 'css'),
                    homePage='index.html',
                    files=[],
                    subDirs=subDirs)

    with open(htmlFile, 'w', encoding='utf-8') as f:
        f.write(html)

In [3]:
if __name__ == '__main__':
    pc1 = time.perf_counter()
    
    # Read main config
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        try:
            appConfig = json.loads(jsonTxt)
        except:
            logger = Printable()
            logger.logError('Could not parse {}'.format(filename))
            raise
    
    # Process the required year(s)
    processEvents()
    
    # Only refresh the index page if all years have been processed
    if appConfig['History']['Refresh']:
        createIndex()
    
    pc2 = time.perf_counter()
    print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Processing 1998...
All done!

Processing 1999...
All done!

Processing 2000...
All done!

Processing 2001...
All done!

Processing 2002...
All done!

Processing 2003...
All done!

Processing 2004...
All done!

Processing 2005...
All done!

Processing 2006...
All done!

Processing 2007...
All done!

Processing 2008...
All done!

Processing 2009...
All done!

Processing 2010...
All done!

Processing 2011...
All done!

Processing 2012...
All done!

Processing 2013...
All done!

Processing 2014...
All done!

Processing 2015...
All done!

Processing 2016...
All done!

Processing 2017...
All done!

Processing 2018...
All done!

Processing 2019...
All done!

Processing 2020...
All done!

Processing 2021...
All done!

Processing 2022...
All done!

Processing 2023...
INFO: 606 runs by 48 participants down course S1 on 20231007 - 11:03:37 to 15:09:25
INFO: 423 runs by 45 participants down course S1 on 20231010 - 10:02:22 to 16:00:41
INFO: 1044 runs by 68 participants down course S1 on 20231011 -

## All Done!